In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings

from datetime import datetime
from ta.trend import ema_indicator, stc, macd, trix, kst
from ta.volatility import bollinger_hband, bollinger_lband, ulcer_index
from ta.momentum import kama, ppo, roc, rsi, stochrsi, tsi


# Suppress warnings to keep the output clean
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
BTC = "BTC-USD"
ETH = "ETH-USD"
SPDR = "SPY"
END_DATE = datetime.today().strftime("%Y-%m-%d")

In [3]:
stocks = yf.Ticker(
    BTC
)

with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore')
    btc = stocks.history(start='2022-09-12', end=END_DATE, interval='1h').drop(columns=['Dividends', 'Stock Splits']).reset_index()
    btc = btc.rename(columns={'Datetime': 'Date'})
    btc_old = pd.read_parquet("../data/btc-2015-2023.pq").drop(columns=['Volume USD'])
    btc_old['Date'] = pd.to_datetime(btc_old['Date']).dt.tz_localize('UTC')
    btc_old = btc_old.loc[btc_old['Date'] < pd.to_datetime("2022-09-12 00:00:00+00:00")]

    btc_2019 = pd.read_parquet("../data/btc-2019.pq")
    btc_2019 = btc_2019.rename(columns={
        col: col.capitalize() for col in btc_2019.columns
    })
    btc_2019['Date'] = pd.to_datetime(btc_2019['Date']).dt.tz_localize('UTC')

    btc_2018 = pd.read_parquet("../data/btc-2018.pq")
    btc_2018 = btc_2018.rename(columns={
        col: col.capitalize() for col in btc_2018.columns
    })
    btc_2018['Date'] = pd.to_datetime(btc_2018['Date']).dt.tz_localize('UTC')


    btc_old = pd.concat([btc_old, btc_2018, btc_2019])

    def get_open(x: pd.Series):
        return x[x.index == np.min(x.index)]

    mapper = {
        'Open': get_open,
    }
    btc_old_processed = btc_old.sort_values(by='Date').reset_index(drop=True).groupby(pd.Grouper(key='Date', freq='H')).agg(mapper).reset_index()
    
    btc = pd.concat([btc, btc_old_processed], axis=0).sort_values(by='Date').reset_index(drop=True)[['Date', 'Open']]
    btc = btc.loc[(btc['Date'].dt.day_of_week == 0) & (btc['Date'].dt.hour == 20)]

# Trend indicators
btc['ema_12'] = ema_indicator(btc['Open'])
btc['ema_26'] = ema_indicator(btc['Open'], window=26)
btc['stc'] = stc(btc['Open'])
btc['macd'] = macd(btc['Open'])
btc['trix'] = trix(btc['Open'])
btc['kst'] = kst(btc['Open'])

# Volatility indicators
btc['bollinger_high'] = bollinger_hband(btc['Open'])
btc['bollinger_low'] = bollinger_lband(btc['Open'])
btc['ulcer_index'] = ulcer_index(btc['Open'])

# Momentum indicators
btc['kama'] = kama(btc['Open'])
btc['ppo'] = ppo(btc['Open'])
btc['roc'] = roc(btc['Open'])
btc['rsi'] = rsi(btc['Open'])
btc['stochrsi'] = stochrsi(btc['Open'])
btc['tsi'] = tsi(btc['Open'])

btc['target'] = ((btc['Open'].shift(-1).fillna(np.nan).values - btc['Open'].values)/btc['Open'].values) * 100
btc = btc.dropna()
btc

,Date,Open,ema_12,ema_26,stc,macd,trix,kst,bollinger_high,bollinger_low,ulcer_index,kama,ppo,roc,rsi,stochrsi,tsi,target
13039,2017-04-03 20:00:00+00:00,1150.01,1054.627396,932.669224,8.676040,121.958171,2.257454,611.356624,1289.738739,637.926261,10.532655,1029.443118,13.076251,28.810807,63.177672,0.206836,34.140221,5.186042
13207,2017-04-10 20:00:00+00:00,1209.65,1078.477027,953.186319,33.676040,125.290708,2.244671,621.496232,1312.690288,662.245712,10.637247,1042.099513,13.144409,45.219573,65.584572,0.375443,33.579181,-1.501261
13375,2017-04-17 20:00:00+00:00,1191.49,1095.863638,970.838443,56.507030,125.025195,2.224012,615.668817,1325.060817,695.756183,10.326580,1047.787912,12.878064,29.681752,64.208310,0.316594,32.679999,5.425979
13543,2017-04-24 20:00:00+00:00,1256.14,1120.521540,991.971892,73.088020,128.549648,2.208783,636.027354,1344.438786,726.566214,9.188764,1060.917850,12.959001,36.189082,66.873409,0.430554,32.851344,14.941806
13711,2017-05-01 20:00:00+00:00,1443.83,1170.261303,1025.442863,83.961263,144.818440,2.235581,676.872454,1402.345386,735.044614,8.848886,1096.748744,14.122527,41.274951,73.129046,0.698048,35.341176,14.168566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75619,2024-05-27 20:00:00+00:00,69200.929688,64546.409393,57227.502322,53.125206,7318.907071,2.539792,881.704973,81836.749846,38347.028279,7.274278,63898.331322,12.789143,2.56459,65.557535,0.194498,34.469099,-0.027287
75787,2024-06-03 20:00:00+00:00,69182.046875,65259.584391,58113.024141,70.313009,7146.560250,2.440202,859.483347,81988.315043,40818.294332,7.349231,63957.100991,12.297691,-3.917275,65.526621,0.193234,33.948874,0.427981
75955,2024-06-10 20:00:00+00:00,69478.132812,65908.591840,58954.884042,82.031939,6953.707798,2.340451,840.015365,81177.653172,44598.044874,7.408726,63984.037777,11.794965,3.648718,65.798963,0.204373,33.598787,-4.094589
76123,2024-06-17 20:00:00+00:00,66633.289062,66020.083720,59523.654785,41.015970,6496.428936,2.228526,794.096468,79964.049310,48174.914753,7.648734,64070.475618,10.914029,-6.141849,60.826736,0.001425,31.893367,-10.623631
